# Import the text

In [10]:
import pandas as pd
json = pd.read_json("webhose_netflix-Copy1.json",lines=True)

In [133]:
text=json['text'][99]
text

'Carole Baskin has some news to share with all the "cool cats and kittens."\nThe founder and CEO of Florida sanctuary Big Cat Rescue received a win Monday. An Oklahoma federal judge has handed over ownership of Greater Wynnewood Exotic Animal Park, the backdrop for Netflix\'s incredibly popular docuseries "Tiger King: Murder, Mayhem and Madness."\nThe true-crime series, released in March, explores the life of Joe Exotic (born Joseph Schreibvogel), a man with an affinity for big cats, blonde mullets, guns and explosives. "Tiger King" examines Joe\'s life at the park (referred to as the G.W. Zoo) and his hatred for Baskin. Joe deemed her a threat to his livelihood because of her advocacy of the Big Cat Public Safety Act, which aims to abolish ownership of big cats as pets and the practice of cub petting.\nJoe\'s malice for Baskin turned to an actual punishable offense when he allegedly hired someone to kill her. He allegedly paid zoo handyman Allen Glover $3,000 to kill Baskin. Glover de

# 1. Extract named entities and visualize the labeled entities with spaCy model "en_core_web_sm"
## 1.1 Named entities list

In [287]:
import spacy
nlp1 = spacy.load("en_core_web_sm")

In [288]:
doc=nlp1(text)

In [289]:
[(ent, ent.label_) for ent in doc.ents]

[(Florida, 'GPE'),
 (Big Cat Rescue, 'ORG'),
 (Monday, 'DATE'),
 (Oklahoma, 'GPE'),
 (Greater Wynnewood Exotic Animal Park, 'ORG'),
 (Netflix, 'ORG'),
 (Tiger King: Murder, Mayhem, 'WORK_OF_ART'),
 (Madness, 'GPE'),
 (March, 'DATE'),
 (Joe Exotic, 'PERSON'),
 (Joseph Schreibvogel, 'PERSON'),
 (Joe, 'PERSON'),
 (the G.W. Zoo, 'ORG'),
 (Baskin, 'PERSON'),
 (Joe, 'PERSON'),
 (the Big Cat Public Safety Act, 'ORG'),
 (Joe, 'PERSON'),
 (Baskin, 'PERSON'),
 (Allen Glover, 'PERSON'),
 (3,000, 'MONEY'),
 (Baskin, 'PERSON'),
 (Joe, 'PERSON'),
 (Joe, 'PERSON'),
 (2018, 'DATE'),
 (19, 'CARDINAL'),
 (five, 'CARDINAL'),
 (January, 'DATE'),
 (22 years, 'DATE'),
 (U.S., 'GPE'),
 (Scott L. Palk, 'PERSON'),
 (Big Cat Rescue Corp., 'ORG'),
 (Monday, 'DATE'),
 (Baskin, 'PERSON'),
 ($1 million, 'MONEY'),
 (Joe, 'PERSON'),
 (Monday, 'DATE'),
 (Joe, 'PERSON'),
 (the Associated Press, 'ORG'),
 (120 days, 'DATE'),
 (Baskin, 'PERSON'),
 (Tuesday, 'DATE'),
 (Big Cat Rescue's, 'ORG'),
 (Jeff Lowe, 'PERSON'),
 (Jo

In [85]:
from spacy import displacy
displacy.render(doc,'ent')

# 2. Extract Company/Organization and Geo entities with Google Cloud NL
## 2.1 Company/Organization and Geo entities list

In [36]:
# Imports the Google Cloud client library
from google.cloud import language_v1

In [37]:
# Instantiates a client
client = language_v1.LanguageServiceClient()

document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
response=client.analyze_entities(request={"document": document})


In [373]:
for entity in response.entities:
    entity_type=language_v1.Entity.Type(entity.type).name
    if entity_type in ['PERSON','ORGANIZATION','LOCATION']:
        print((entity.name, entity_type))

('Joe Exotic', 'PERSON')
('Carole Baskin', 'PERSON')
('Big Cat Rescue', 'ORGANIZATION')
('kittens', 'PERSON')
('judge', 'PERSON')
('founder', 'PERSON')
('sanctuary', 'LOCATION')
('CEO', 'PERSON')
('Scott L. Palk', 'PERSON')
('Allen Glover', 'PERSON')
('Greater Wynnewood Exotic Animal Park', 'LOCATION')
('someone', 'PERSON')
('Florida', 'LOCATION')
('Oklahoma', 'LOCATION')
('Netflix', 'ORGANIZATION')
('park', 'LOCATION')
('zoo', 'LOCATION')
('man', 'PERSON')
('Tiger King', 'PERSON')
('Associated Press', 'ORGANIZATION')
('Jeff Lowe', 'PERSON')
('Kate McKinnon', 'PERSON')
('prison', 'LOCATION')
('mother', 'PERSON')
('G.W.', 'PERSON')
('homes', 'LOCATION')
('animal welfare organizations', 'ORGANIZATION')
('facilities', 'LOCATION')
('viewers', 'PERSON')
('U.S.', 'LOCATION')
('zoo owner', 'PERSON')
('Nicolas Cage', 'PERSON')
('homes', 'LOCATION')
('USA TODAY', 'ORGANIZATION')
('Leif Reigstad', 'PERSON')
('Texas Monthly', 'ORGANIZATION')
('CBS', 'ORGANIZATION')


## 2.2 Visualize the Company/Organization and Geo entities with displacy

In [343]:
#save the entity names in google could NL results
PERSON=[]
ORGANIZATION=[]
LOCATION=[]
for entity in response.entities:
    entity_type=language_v1.Entity.Type(entity.type).name
    if entity_type =='PERSON':
        PERSON.append(entity.name)
    if entity_type =='ORGANIZATION':
        ORGANIZATION.append(entity.name)
    if entity_type =='LOCATION':
        LOCATION.append(entity.name)
print(PERSON,'\n',ORGANIZATION,'\n',LOCATION)

['Joe Exotic', 'Carole Baskin', 'kittens', 'judge', 'founder', 'CEO', 'Scott L. Palk', 'Allen Glover', 'someone', 'man', 'Tiger King', 'Jeff Lowe', 'Kate McKinnon', 'mother', 'G.W.', 'viewers', 'zoo owner', 'Nicolas Cage', 'Leif Reigstad'] 
 ['Big Cat Rescue', 'Netflix', 'Associated Press', 'animal welfare organizations', 'USA TODAY', 'Texas Monthly', 'CBS'] 
 ['sanctuary', 'Greater Wynnewood Exotic Animal Park', 'Florida', 'Oklahoma', 'park', 'zoo', 'prison', 'homes', 'facilities', 'U.S.', 'homes']


In [364]:
#locate the entity names in spacy doc
import re
doc=nlp1(text)
ents=[]
for name in PERSON:
    place = re.search(name, doc.text)
    if place != None:
        ents.append({'start': place.start(), 'end': place.end(), 'label': 'PERSON'})
for name in ORGANIZATION:
    place = re.search(name, doc.text)
    if place != None:
        ents.append({'start': place.start(), 'end': place.end(), 'label': 'ORGANIZATION'})
for name in LOCATION:
    place = re.search(name, doc.text)
    if place != None:
        ents.append({'start': place.start(), 'end': place.end(), 'label': 'LOCATION'})
ents

[{'start': 409, 'end': 419, 'label': 'PERSON'},
 {'start': 0, 'end': 13, 'label': 'PERSON'},
 {'start': 65, 'end': 72, 'label': 'PERSON'},
 {'start': 174, 'end': 179, 'label': 'PERSON'},
 {'start': 79, 'end': 86, 'label': 'PERSON'},
 {'start': 91, 'end': 94, 'label': 'PERSON'},
 {'start': 1332, 'end': 1345, 'label': 'PERSON'},
 {'start': 948, 'end': 960, 'label': 'PERSON'},
 {'start': 896, 'end': 903, 'label': 'PERSON'},
 {'start': 450, 'end': 453, 'label': 'PERSON'},
 {'start': 305, 'end': 315, 'label': 'PERSON'},
 {'start': 1861, 'end': 1870, 'label': 'PERSON'},
 {'start': 2731, 'end': 2744, 'label': 'PERSON'},
 {'start': 1617, 'end': 1623, 'label': 'PERSON'},
 {'start': 587, 'end': 591, 'label': 'PERSON'},
 {'start': 2325, 'end': 2332, 'label': 'PERSON'},
 {'start': 2414, 'end': 2423, 'label': 'PERSON'},
 {'start': 2366, 'end': 2378, 'label': 'PERSON'},
 {'start': 2542, 'end': 2555, 'label': 'PERSON'},
 {'start': 116, 'end': 130, 'label': 'ORGANIZATION'},
 {'start': 264, 'end': 271,

In [365]:
ex = {
    "text": text, 
    "ents": ents, 
    "title": None,
}

In [368]:
colors = {'PERSON': '#EBDBFD', 'ORGANIZATION': '#FECEDE','LOCATION': '#BABACD'}

In [369]:
displacy.render(ex, style="ent", manual=True,options={'colors': colors})

# 3. Train Entity Recognizer
## 3.1 Create a traning data using text in step 1

In [176]:
text=text.strip()
segments = text.split("\n")
segments

['Carole Baskin has some news to share with all the "cool cats and kittens."',
 'The founder and CEO of Florida sanctuary Big Cat Rescue received a win Monday. An Oklahoma federal judge has handed over ownership of Greater Wynnewood Exotic Animal Park, the backdrop for Netflix\'s incredibly popular docuseries "Tiger King: Murder, Mayhem and Madness."',
 'The true-crime series, released in March, explores the life of Joe Exotic (born Joseph Schreibvogel), a man with an affinity for big cats, blonde mullets, guns and explosives. "Tiger King" examines Joe\'s life at the park (referred to as the G.W. Zoo) and his hatred for Baskin. Joe deemed her a threat to his livelihood because of her advocacy of the Big Cat Public Safety Act, which aims to abolish ownership of big cats as pets and the practice of cub petting.',
 "Joe's malice for Baskin turned to an actual punishable offense when he allegedly hired someone to kill her. He allegedly paid zoo handyman Allen Glover $3,000 to kill Baskin. 

In [183]:
def test_model(model,text):
    doc=nlp1(text)
    results=[]
    entities=[]
    for ent in doc.ents:
        if ent.label_ in ['PERSON','ORG','GPE']:
            entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities)>0:
        results=[text,{"entities":entities}]
    return (results)

In [184]:
TRAIN_DATA=[]
for segment in segments:
        segment = segment.strip()
        results = test_model(nlp1,segment)
        if len(results)>0:
            TRAIN_DATA.append(results)
print(TRAIN_DATA)

[['The founder and CEO of Florida sanctuary Big Cat Rescue received a win Monday. An Oklahoma federal judge has handed over ownership of Greater Wynnewood Exotic Animal Park, the backdrop for Netflix\'s incredibly popular docuseries "Tiger King: Murder, Mayhem and Madness."', {'entities': [(23, 30, 'GPE'), (41, 55, 'ORG'), (82, 90, 'GPE'), (134, 170, 'ORG'), (189, 196, 'ORG'), (261, 268, 'GPE')]}], ['The true-crime series, released in March, explores the life of Joe Exotic (born Joseph Schreibvogel), a man with an affinity for big cats, blonde mullets, guns and explosives. "Tiger King" examines Joe\'s life at the park (referred to as the G.W. Zoo) and his hatred for Baskin. Joe deemed her a threat to his livelihood because of her advocacy of the Big Cat Public Safety Act, which aims to abolish ownership of big cats as pets and the practice of cub petting.', {'entities': [(63, 73, 'PERSON'), (80, 99, 'PERSON'), (198, 201, 'PERSON'), (237, 249, 'ORG'), (270, 276, 'PERSON'), (278, 281, 'P

## 3.2 Train the Model

In [74]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [250]:
import random
from spacy.training import Example
import matplotlib.pyplot as plt

In [281]:
def traindata(TRAIN_DATA, literations):
    nlp = spacy.blank('en')
    if 'ner' not in nlp.pipe_names:
        nlp.add_pipe('ner',last=True)
        
    for _,annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            
    with nlp.select_pipes(enable=["ner"]):
        optimizer = nlp.begin_training()
        for itn in range(literations):
            losses = {}
            texts, annotations = zip(*TRAIN_DATA)
            example = []
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            #set the drop to prevent overlifting
            nlp.update(example, drop=0.2, losses=losses, sgd=optimizer)
            print(losses['ner'])
            
    return(nlp)

In [282]:
#print the losses
nlp2=traindata(TRAIN_DATA,100)

500.4999945163727
493.06575763225555
481.7743316888809
466.23500245809555
441.3229191303253
403.38707053661346
337.9149337410927
249.18785132467747
135.39589121937752
83.462157596834
75.65052289326559
77.65216793036961
148.18848521954715
72.86052423375077
70.60195848567673
66.48063503716185
61.08467254133575
58.26917807522477
52.627534991632274
48.04958494556922
45.926698790255614
45.63607482803127
42.384912314637404
41.86660040575043
40.887567332710404
38.34884867739041
37.1581167344205
35.91741014174825
33.182331843479645
33.12371178170865
70.99805666781532
29.784602282240257
29.222899916992162
31.684535496456984
29.19361650372026
31.16418606665544
60.220939659534636
142.63616196717442
66.82497640317894
272.0926535647434
177.7883529484029
123.2288896269737
175.65838033147588
115.57291673491355
81.5610446188623
66.68806350893169
59.914339688490145
53.040841927686415
51.05537622873089
50.78732362387382
48.48210647364613
46.17737991565082
41.5755802977219
38.27439105412668
33.0351586338

## 3.3 Apply the model to another text

In [191]:
text2=json['text'][999]
text2

'Carole Baskin granted control of Joe Exotic\'s zoo, Jeff Lowe must vacate in 120 days 2020-06-02 08:14 by Bashiera Parker (@bparks_) Carol Baskin in \'Tiger King: Murder, Mayhem and Madness\'. (Netflix) Carole Baskin has assumed ownership of arch-rival, Joe Exotic\'s zoo after it was found to have been fraudulently transferred to his mother years ago. Jeff Lowe, another of Joe\'s notorious enemies, has been running the zoo since Joe\'s been incarcerated for hiring someone to kill Carole Baskin. Now, Jeff has 120 days to vacate the property and hand everything over to Big Cat Rescue Corp – Carole\'s animal rescue sanctuary.\nAnd the plot thickens...\nOn Monday Fox 25 reported that Judge Scott Palk ordered the G.W. Exotic Animal Memorial Park in Wynnewood, Oklahoma, which once belonged to Joseph Maldonado-Passage, aka, Joe Exotic, be handed over to Big Cat Rescue Corp – Carole Baskin\'s animal rescue sanctuary.\nIn case you haven\'t yet streamed Tiger King: Murder, Mayhem and Madness , 

In [286]:
#print the entity list
doc=nlp2(text2)
[(ent, ent.label_) for ent in doc.ents]

[(Carole Baskin granted control, 'ORG'),
 (Joe Exotic, 'PERSON'),
 (Jeff Lowe, 'PERSON'),
 (Bashiera Parker, 'PERSON'),
 (Baskin, 'PERSON'),
 (Madness, 'GPE'),
 (Netflix, 'ORG'),
 (Carole Baskin has assumed, 'ORG'),
 (Joe Exotic, 'PERSON'),
 (Jeff Lowe, 'PERSON'),
 (Joe, 'PERSON'),
 (Joe, 'PERSON'),
 (Carole Baskin. Now, 'ORG'),
 (Jeff has, 'PERSON'),
 (Big Cat Rescue Corp, 'ORG'),
 (Carole, 'PERSON'),
 (On Monday, 'PERSON'),
 (Scott Palk, 'PERSON'),
 (the G.W. Exotic Animal Memorial, 'ORG'),
 (Wynnewood, Oklahoma, 'ORG'),
 (Joseph Maldonado, 'PERSON'),
 (Joe Exotic, 'PERSON'),
 (Big Cat Rescue Corp, 'ORG'),
 (Carole Baskin's animal, 'ORG'),
 (Madness, 'GPE'),
 (Joe Exotic, 'PERSON'),
 (Oklahoma with, 'PERSON'),
 (Carole Baskin. We, 'ORG'),
 (Jeff Lowe, 'PERSON'),
 (Joe, 'PERSON'),
 (Carole Baskin and Big Cat Rescue, 'ORG'),
 (Joe, 'PERSON'),
 (Shirley M., 'PERSON'),
 (Jeff Lowe, 'PERSON'),
 (Entertainment Weekly :, 'ORG'),
 (Jeff Lowe, 'PERSON')]

In [283]:
#visualize the results
displacy.render(doc,'ent')